In [2]:
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, BigInteger, Sequence, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import os
from libv3.utils import *
import pandas as pd
import json
import shutil

# Define the base for the declarative model
Base = declarative_base()

# Define the Event class which will be mapped to the events table in the database
class Event(Base):
    __tablename__ = 'events'
    
    id: Mapped[int] = mapped_column(primary_key=True)
    
    name: Mapped[int] = mapped_column(nullable=False)
    timestamp: Mapped[int] = mapped_column(BigInteger, nullable=False)
    file_number: Mapped[int] = mapped_column(ForeignKey("file_config.id")) 

    config: Mapped["File_config"] = relationship(back_populates="events")
    


class File_config(Base):
    __tablename__ = 'file_config'

    id: Mapped[int] = mapped_column(primary_key=True)
    unique_name: Mapped[str] = mapped_column(String(50), nullable=False, unique=True)
    code_base: Mapped[str] = mapped_column(String(50), nullable=False)
    version: Mapped[int] = mapped_column(Integer, nullable=False)
    behaviour: Mapped[str] = mapped_column(String(50), nullable=False)
    trial_num: Mapped[int] = mapped_column(Integer, nullable=False)

    events: Mapped[List["Event"]] = relationship(back_populates="config")


def get_or_create(code_base, version, behaviour, trial_num):
    '''
    Function to get or create a new file_config entry in
    the database. This function will return the existing
    entry if it exists, otherwise it will create a new entry
    and return it.

    Args:
        code_base (str): The code base used to generate the data
        version (int): The version of the code base
        behaviour (str): The behaviour being tested
        trial_num (int): The trial number of the data

    Returns:
        File_config: The file_config entry
        bool: False if the entry was created, True if the entry already existed
    '''
    with Session() as session:
        instance = session.query(File_config).filter_by(unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}').first()
        if instance:
            return instance, True  # Return existing entry
        else:
            new_config = File_config(code_base=code_base,version=version, behaviour=behaviour, trial_num=trial_num, unique_name=f'{code_base}_{version}_{behaviour}_{trial_num}')
            session.add(new_config)
            session.commit()
            return new_config, False  # New config added
        
def check_event_exists(name, timestamp, unique_name):
    '''
    Function to check if an event already exists in the database

    Args:
        name (str): The name of the event
        timestamp (int): The timestamp of the event
        file_number (int): The file number of the event

    Returns:
        bool: True if the event exists, False if it does not
    '''
    with Session() as session:
        file_number = session.query(File_config).filter_by(unique_name=unique_name).first().id
        instance = session.query(Event).filter_by(name=name, timestamp=timestamp, file_number=file_number).first()
        if instance:
            return True
        else:
            return False



/var/folders/nw/443sknns78l256x46fjj6r0c0000gn/T/ipykernel_2062/3485431635.py:17: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [114]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config('mamba2_config')   ### config stored in libv3/exp_config.txt
TRIAL = 3

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trial{TRIAL}'

print(log_path)

#### file to display
trace_file = 0

print('file number:', trace_file)

CODE: mamba2
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/mamba2/single_thread/version_3/faulty_data/trial3
file number: 0


In [115]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

[]
['../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace0', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace1', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace10', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace11', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace2', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace3', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace4', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace5', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace6', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace7', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace8', '../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/trace9']
['../trace_data/mamba2/single_thread/version_3/faulty_data/trial3/varlist0', '../trace_data/mamba2/single_

### Combine files from trials (only to create trial files)

In [116]:
# #### combine files
# paths_traces.sort(key=lambda x: int(x.split('/')[-1][5:]))
# last_file_num = int(paths_traces[-1].split('/')[-1][5:])
# all_content = []
# for trace_path in paths_traces:
#     content = read_traces(trace_path)   
#     all_content.extend(content)
#     # print(content)
#     # break
    
# path_to_save = os.path.dirname(os.path.dirname(trace_path))
# with open(path_to_save+f'/trace_trial{TRIAL}', 'w') as f:
#     json.dump(all_content, f)
#     print('Saved trace:', path_to_save+f'/trace_trial{TRIAL}')


# varlist_content = read_json(os.path.join(os.path.dirname(trace_path),f'varlist{last_file_num}'))
# print(varlist_content)
# with open(path_to_save+f'/varlist_trial{TRIAL}.json', 'w') as f:
#     json.dump(varlist_content, f)



# # varlist_content = []
# # for varlist_path in varlist_path:
# #     content = read_json(varlist_path)
# #     varlist_content.extend(content)
# #     varlist_content = list(set(varlist_content)) ### remove duplicates


Saved trace: ../trace_data/mamba2/single_thread/version_3/faulty_data/trace_trial3
{'0-0-0-CONNECTION_CO2': 11, '0-0-0-var': 61, '0-0-0-BMP': 38, '0-0-0-CONNECTION_O2': 13, '0-0-measure_am1-0': 64, '0-0-0-THRESHOLD_LIMITS': 43, '0-0-0-msg_interval': 28, '0-0-0-O2_ADRR': 3, '0-0-0-AM2301_1_ADRR': 6, '0-0-measure_scd30-0': 58, '0-0-measure_am4-0': 67, '0-0-cb_lora-board_id': 71, '0-0-cb_30-cb_30_done': 68, '0-0-0-cb_30_done': 26, '0-0-0-SPI_BUS': 33, '0-0-measure_co-0': 60, '0-0-0-am_temp': 23, '0-0-0-FUNC_VAR': 46, '0-0-0-CONNECTION_CO': 12, '0-0-0-timer1': 48, '0-0-0-CONNECTION_A1': 15, '0-0-0-CONNECTION_A2': 16, '0-0-0-CONNECTION_A3': 17, '0-0-0-CONNECTION_A4': 18, '0-0-0-timer0': 47, '0-0-0-AM2301_2_ADRR': 7, '0-0-0-MCP_O2': 37, '0-0-0-SENSOR_DATA': 49, '0-0-0-SCD30_ADRR': 5, '0-0-0-lora': 34, '0-0-0-que': 25, '0-0-measure_am2-0': 65, '0-0-add_to_que-que': 69, '0-0-cb_retrans-cb_retrans_done': 73, '0-0-0-func_call': 57, '0-0-0-LIMITS_BROKEN': 55, '0-0-cb_lora-timestamp': 72, '0-0-0-C

### Load the existing files

In [117]:
### load combined trace file and varlist
all_content = read_traces(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/trace_trial{TRIAL}')
varlist_content = read_json(f'/Users/saurabh/Documents/Nextcloud/work/Phd tasks/diagnosis tool/trace_data/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}/varlist_trial{TRIAL}.json')

In [118]:
len(all_content)

11132

In [119]:
len(varlist_content)

74

In [120]:
path_to_content = os.path.dirname(log_path)+f'/trace_trial{TRIAL}'
content_towrite = read_traces(path_to_content)
# print(content_towrite)
# Create an SQLite database (or connect to it if it already exists)
database_url = 'sqlite:///events.db'
engine = create_engine(database_url, echo=True)

# Create all tables in the database
Base.metadata.create_all(engine)

# # Create a configured "Session" class
Session = sessionmaker(bind=engine)

config_settings, config_exists = get_or_create(code_base=CODE, version=VER, behaviour=BEHAVIOUR, trial_num=TRIAL)
print(config_settings, config_exists)

if not config_exists:
    with Session.begin() as session:
        for event_name, event_timestamp in all_content:
            print(event_name, event_timestamp)

            # event_exists = check_event_exists(event_name, event_timestamp, f'{CODE}_{VER}_{BEHAVIOUR}_{TRIAL}')
            # if event_exists:
            #     print(f"Event {event_name} at timestamp {event_timestamp} already exists in the database")
            #     break

            event = Event(name=event_name, timestamp=event_timestamp, config=config_settings)
            session.add(event)
else:
    print("Config already exists in the database")


2024-07-25 15:32:51,234 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-25 15:32:51,236 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("events")
2024-07-25 15:32:51,236 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-25 15:32:51,239 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("file_config")
2024-07-25 15:32:51,241 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-25 15:32:51,242 INFO sqlalchemy.engine.Engine COMMIT


2024-07-25 15:32:51,248 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-25 15:32:51,251 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_config.unique_name AS file_config_unique_name, file_config.code_base AS file_config_code_base, file_config.version AS file_config_version, file_config.behaviour AS file_config_behaviour, file_config.trial_num AS file_config_trial_num 
FROM file_config 
WHERE file_config.unique_name = ?
 LIMIT ? OFFSET ?
2024-07-25 15:32:51,252 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ('mamba2_3_faulty_data_3', 1, 0)
2024-07-25 15:32:51,256 INFO sqlalchemy.engine.Engine INSERT INTO file_config (unique_name, code_base, version, behaviour, trial_num) VALUES (?, ?, ?, ?, ?)
2024-07-25 15:32:51,256 INFO sqlalchemy.engine.Engine [generated in 0.00062s] ('mamba2_3_faulty_data_3', 'mamba2', 3, 'faulty_data', 3)
2024-07-25 15:32:51,258 INFO sqlalchemy.engine.Engine COMMIT
<__main__.File_config object at 0x1620b37f0> False
0 10
1 1

In [113]:
#### query the database


with Session.begin() as session:
    results = session.query(File_config).all()
    for i in results:
        print(i.code_base, i.version, i.behaviour, i.trial_num, i.id)

2024-07-25 15:16:24,963 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-25 15:16:24,966 INFO sqlalchemy.engine.Engine SELECT file_config.id AS file_config_id, file_config.unique_name AS file_config_unique_name, file_config.code_base AS file_config_code_base, file_config.version AS file_config_version, file_config.behaviour AS file_config_behaviour, file_config.trial_num AS file_config_trial_num 
FROM file_config
2024-07-25 15:16:24,966 INFO sqlalchemy.engine.Engine [cached since 252.8s ago] ()
theft_protection 3 normal 1 3
theft_protection 3 faulty_data 1 4
theft_protection 3 faulty_data 2 5
theft_protection 3 faulty_data 3 6
mamba 3 normal 1 7
mamba 3 faulty_data 1 8
mamba 3 faulty_data 3 10
mamba 3 faulty_data 4 11
mamba 3 faulty_data 2 12
mamba 3 normal 2 13
mamba2 3 normal 1 14
mamba2 3 normal 2 15
mamba2 3 faulty_data 1 16
mamba2 3 faulty_data 2 17
mamba2 3 faulty_data 4 19
2024-07-25 15:16:24,968 INFO sqlalchemy.engine.Engine COMMIT


In [112]:
#### delete entries

# to_delete = 20

with Session.begin() as session:
    results = session.query(Event).all()
    file_deleted = False
    for i in results:
        if i.file_number == to_delete:
            print(i.name, i.timestamp, i.file_number, 'deleted')
            session.delete(i)
            file_deleted = True

    if not file_deleted:
        print('File config does not exist')

with Session.begin() as session:
    results = session.query(File_config).all()
    event_deleted = False
    for i in results:
        if i.id == to_delete:
            print(i.code_base, i.version, i.behaviour, i.trial_num, i.id, 'deleted')
            session.delete(i)
            event_deleted = True
            
    if not event_deleted:
        print('Event does not exist')

2024-07-25 15:16:13,094 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2024-07-25 15:16:13,108 INFO sqlalchemy.engine.Engine SELECT events.id AS events_id, events.name AS events_name, events.timestamp AS events_timestamp, events.file_number AS events_file_number 
FROM events
2024-07-25 15:16:13,111 INFO sqlalchemy.engine.Engine [generated in 0.00334s] ()
0 10 20 deleted
1 13 20 deleted
2 14 20 deleted
3 15 20 deleted
4 16 20 deleted
5 18 20 deleted
6 19 20 deleted
7 20 20 deleted
8 21 20 deleted
9 22 20 deleted
10 23 20 deleted
11 24 20 deleted
12 25 20 deleted
13 26 20 deleted
14 28 20 deleted
15 29 20 deleted
16 30 20 deleted
17 31 20 deleted
18 32 20 deleted
19 33 20 deleted
20 35 20 deleted
21 36 20 deleted
22 37 20 deleted
23 38 20 deleted
24 39 20 deleted
25 40 20 deleted
26 42 20 deleted
27 43 20 deleted
28 44 20 deleted
29 45 20 deleted
30 47 20 deleted
31 48 20 deleted
30 51 20 deleted
32 54 20 deleted
33 56 20 deleted
34 66 20 deleted
35 91 20 deleted
36 118 20 deleted
37 143 20 deleted
38 304 20 deleted
39 306 20 deleted
40 307 20 deleted
41 30